In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics  import f1_score, confusion_matrix, make_scorer
from sklearn.pipeline import make_pipeline
from sklearn import metrics

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/Colab Notebooks/230219-DataHackathon


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Colab Notebooks/230219-DataHackathon


In [ ]:
cyclica = pd.read_csv("230219-cyclicaNoEntry.csv")

In [ ]:
cyclica.columns

Index(['Unnamed: 0', 'annotation_sequence', 'feat_A', 'feat_C', 'feat_D',
       'feat_E', 'feat_F', 'feat_G', 'feat_H', 'feat_I', 'feat_K', 'feat_L',
       'feat_M', 'feat_N', 'feat_P', 'feat_Q', 'feat_R', 'feat_S', 'feat_T',
       'feat_V', 'feat_W', 'feat_Y', 'annotation_atomrec', 'feat_PHI',
       'feat_PSI', 'feat_TAU', 'feat_THETA', 'feat_BBSASA', 'feat_SCSASA',
       'feat_pLDDT', 'feat_DSSP_H', 'feat_DSSP_B', 'feat_DSSP_E',
       'feat_DSSP_G', 'feat_DSSP_I', 'feat_DSSP_T', 'feat_DSSP_S',
       'feat_DSSP_6', 'feat_DSSP_7', 'feat_DSSP_8', 'feat_DSSP_9',
       'feat_DSSP_10', 'feat_DSSP_11', 'feat_DSSP_12', 'feat_DSSP_13',
       'coord_X', 'coord_Y', 'coord_Z', 'y_Ligand'],
      dtype='object')

In [ ]:
X, y = cyclica.drop(labels="y_Ligand", axis=1), cyclica["y_Ligand"]
X_train, X_test, y_train, y_test = \
    train_test_split(X, y,
                    train_size=4000,
                    test_size=1000,
                    random_state=226)

In [ ]:
X.columns

Index(['Unnamed: 0', 'annotation_sequence', 'feat_A', 'feat_C', 'feat_D',
       'feat_E', 'feat_F', 'feat_G', 'feat_H', 'feat_I', 'feat_K', 'feat_L',
       'feat_M', 'feat_N', 'feat_P', 'feat_Q', 'feat_R', 'feat_S', 'feat_T',
       'feat_V', 'feat_W', 'feat_Y', 'annotation_atomrec', 'feat_PHI',
       'feat_PSI', 'feat_TAU', 'feat_THETA', 'feat_BBSASA', 'feat_SCSASA',
       'feat_pLDDT', 'feat_DSSP_H', 'feat_DSSP_B', 'feat_DSSP_E',
       'feat_DSSP_G', 'feat_DSSP_I', 'feat_DSSP_T', 'feat_DSSP_S',
       'feat_DSSP_6', 'feat_DSSP_7', 'feat_DSSP_8', 'feat_DSSP_9',
       'feat_DSSP_10', 'feat_DSSP_11', 'feat_DSSP_12', 'feat_DSSP_13',
       'coord_X', 'coord_Y', 'coord_Z'],
      dtype='object')

In [ ]:
def true_pos_rate (y_true, y_pred):
  TP = confusion_matrix(y_true, y_pred, normalize = 'true')[1,1]
  return TP

TP_scorer = make_scorer(true_pos_rate, greater_is_better = True)
# true_pos_rate([1,1,0,0,1,1], [0,1,1,0,1,0])

In [ ]:
RF_cv = GridSearchCV(
    estimator = RandomForestClassifier(),
    param_grid= {
        'max_depth' : np.arange(1,5),
        'min_samples_leaf' : np.arange(5,20,5),
        'bootstrap' : [True],
        'oob_score' : [True]
    },
    cv = 10, 
    scoring = TP_scorer,
    return_train_score = True
)

RF_cv.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestClassifier(),
             param_grid={'bootstrap': [True], 'max_depth': array([1, 2, 3, 4]),
                         'min_samples_leaf': array([ 5, 10, 15]),
                         'oob_score': [True]},
             return_train_score=True, scoring=make_scorer(true_pos_rate))

In [ ]:
pd.DataFrame(RF_cv.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_bootstrap,param_max_depth,param_min_samples_leaf,param_oob_score,params,split0_test_score,...,split2_train_score,split3_train_score,split4_train_score,split5_train_score,split6_train_score,split7_train_score,split8_train_score,split9_train_score,mean_train_score,std_train_score
0,0.240426,0.008390,0.013785,0.002957,True,1,5,True,"{'bootstrap': True, 'max_depth': 1, 'min_sampl...",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.315853,0.122892,0.019417,0.019483,True,1,10,True,"{'bootstrap': True, 'max_depth': 1, 'min_sampl...",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.414947,0.289278,0.026121,0.025157,True,1,15,True,"{'bootstrap': True, 'max_depth': 1, 'min_sampl...",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.314804,0.061738,0.015532,0.003289,True,2,5,True,"{'bootstrap': True, 'max_depth': 2, 'min_sampl...",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.280489,0.007924,0.013872,0.002297,True,2,10,True,"{'bootstrap': True, 'max_depth': 2, 'min_sampl...",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.304461,0.054547,0.013291,0.002418,True,2,15,True,"{'bootstrap': True, 'max_depth': 2, 'min_sampl...",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.420438,0.069276,0.019207,0.004197,True,3,5,True,"{'bootstrap': True, 'max_depth': 3, 'min_sampl...",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.404913,0.120704,0.017043,0.006283,True,3,10,True,"{'bootstrap': True, 'max_depth': 3, 'min_sampl...",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.347578,0.054858,0.014547,0.003021,True,3,15,True,"{'bootstrap': True, 'max_depth': 3, 'min_sampl...",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.461694,0.075313,0.017848,0.003837,True,4,5,True,"{'bootstrap': True, 'max_depth': 4, 'min_sampl...",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
RF_cv.scorer_

make_scorer(true_pos_rate)

In [ ]:
RF_cv.best_score_

0.0

In [ ]:
RF_best_gini = RF_cv.best_estimator_.fit(X_train, y_train)
y_prob = RF_best_gini.predict_proba(X_test)
y_pred = RF_best_gini.predict(X_test)

f1_loss = f1_score(y_test, y_pred, average='micro')
accuracy = metrics.accuracy_score(y_test, y_pred) 

conf_mtrx = metrics.confusion_matrix(y_test.tolist(), y_pred.tolist(), normalize= 'true')
true_pos_rate = conf_mtrx[1,1]
true_neg_rate = conf_mtrx[0,0]

fpr, tpr, thresholds = metrics.roc_curve(y_test, y_prob[:, 1], pos_label = 1)
AUC_score = metrics.auc(fpr, tpr)

[f1_loss, accuracy, true_pos_rate, true_neg_rate, AUC_score]


[0.965, 0.965, 0.0, 1.0, 0.7680088823094005]

In [ ]:
## randomForest using entropy
# RFclf_entropy = RandomForestClassifier(max_features= 'sqrt',
#                                        criterion="entropy",
#                                     bootstrap=False,
#                                 min_samples_leaf= 5,
#                                 random_state=226)
# RFclf_entropy = RFclf_entropy.fit(X_train, y_train)

# y_prob = RFclf_entropy.predict_proba(X_test)
# y_pred = RFclf_entropy.predict(X_test)

# f1_loss = f1_score(y_test, y_pred, average='micro')
# accuracy = metrics.accuracy_score(y_test, y_pred) 

# conf_mtrx = metrics.confusion_matrix(y_test.tolist(), y_pred.tolist(), normalize= 'true')
# true_pos_rate = conf_mtrx[1,1]
# true_neg_rate = conf_mtrx[0,0]

# fpr, tpr, thresholds = metrics.roc_curve(y_test, y_prob[:, 1], pos_label = 1)
# AUC_score = metrics.auc(fpr, tpr)

# performance_df.loc["RandomForest_Entropy"] = [f1_loss, accuracy, true_pos_rate, true_neg_rate, AUC_score]
# performance_df
## Quite similar result but better AUC than logistic regression

In [ ]:
## randomForest using log loss
# RFclf_logloss = RandomForestClassifier(max_features= 'sqrt',
#                                        criterion="log_loss",
#                                     bootstrap=False,
#                                 min_samples_leaf= 5,
#                                 random_state=226)
# RFclf_logloss = RFclf_logloss.fit(X_train, y_train)

# y_prob = RFclf_logloss.predict_proba(X_test)
# y_pred = RFclf_logloss.predict(X_test)

# f1_loss = f1_score(y_test, y_pred, average='micro')
# accuracy = metrics.accuracy_score(y_test, y_pred) 

# conf_mtrx = metrics.confusion_matrix(y_test.tolist(), y_pred.tolist(), normalize= 'true')
# true_pos_rate = conf_mtrx[1,1]
# true_neg_rate = conf_mtrx[0,0]

# fpr, tpr, thresholds = metrics.roc_curve(y_test, y_prob[:, 1], pos_label = 1)
# AUC_score = metrics.auc(fpr, tpr)

# performance_df.loc["RandomForest_logloss"] = [f1_loss, accuracy, true_pos_rate, true_neg_rate, AUC_score]
# performance_df
## Quite similar result but better AUC than logistic regression

In [ ]:
# AdaBoost
# grid_param = {
#   'learning_rate' : [0.05, 0.1]
# }

# ada_cv = GridSearchCV(
#     estimator= AdaBoostClassifier(),
#     param_grid=grid_param,
#     cv = 10,
#     scoring = TP_scorer,
#     return_train_score = True
# ).fit(X_train, y_train)


print(f"The best estimator is {ada_cv.best_estimator_}")
print(f"The highest true positive score is {ada_cv.best_score_}")


The best estimator is AdaBoostClassifier(learning_rate=0.05)
The highest true positive score is 0.0


In [ ]:
best_Ada = ada_cv.best_estimator_.fit(X_train, y_train)

y_prob = best_Ada.predict_proba(X_test)
y_pred = best_Ada.predict(X_test)

f1_loss = f1_score(y_test, y_pred, average='micro')
accuracy = metrics.accuracy_score(y_test, y_pred) 

conf_mtrx = metrics.confusion_matrix(y_test.tolist(), y_pred.tolist(), normalize='true')
true_pos_rate = conf_mtrx[1,1]
true_neg_rate = conf_mtrx[0,0]

fpr, tpr, thresholds = metrics.roc_curve(y_test, y_prob[:, 1], pos_label = 1)
AUC_score = metrics.auc(fpr, tpr)

# performance_df.loc["AdaBoost"] = [f1_loss, accuracy, true_pos_rate, true_neg_rate, AUC_score]
# performance_df
## Quite similar result but better AUC than logistic regression

In [ ]:
Grad_perf = {
  "f1_loss":f1_loss, 
  'accuracy': accuracy,
  "true positive rate": true_pos_rate, 
  "true negative rate" : true_neg_rate, 
  "AUC" : AUC_score
}
print(Grad_perf)

{'f1_loss': 0.965, 'accuracy': 0.965, 'true positive rate': 0.0, 'true negative rate': 1.0, 'AUC': 0.7669874167283495}


In [ ]:
## Gradient Boosting Classifier
## No tuning
# GB_naive = GradientBoostingClassifier().fit(X_train, y_train)

def performance_report(estimator, X_test, y_test):
  y_prob = estimator.predict_proba(X_test)
  y_pred = estimator.predict(X_test)

  f1_loss = f1_score(y_test, y_pred, average='micro')
  accuracy = metrics.accuracy_score(y_test, y_pred) 

  conf_mtrx = metrics.confusion_matrix(y_test.tolist(), y_pred.tolist(), normalize='true')
  true_pos_rate = conf_mtrx[1,1]
  true_neg_rate = conf_mtrx[0,0]

  fpr, tpr, thresholds = metrics.roc_curve(y_test, y_prob[:, 1], pos_label = 1)
  AUC_score = metrics.auc(fpr, tpr)

  perf = {
    "f1_loss":f1_loss, 
    'accuracy': accuracy,
    "true positive rate": true_pos_rate, 
    "true negative rate" : true_neg_rate, 
    "AUC" : AUC_score
    }
  
  return perf

print(performance_report(GB_naive, X_test, y_test))

{'f1_loss': 0.962, 'accuracy': 0.962, 'true positive rate': 0.0, 'true negative rate': 0.9968911917098445, 'AUC': 0.8124648408586232}


In [ ]:
# X_train

In [ ]:
## Gradient Boosting Classifier
## Rough tuning
# grid_param['loss'] = ['log_loss', 'deviance', 'exponential']
# grid_param['n_estimators'] = np.arange(100, 210, 20)
# GB_cv = GridSearchCV(
#     estimator= GradientBoostingClassifier(),
#     param_grid=grid_param,
#     cv = 10,
#     scoring = TP_scorer,
#     return_train_score = True
# ).fit(X_train, y_train)



print(f"The best estimator is {GB_cv.best_estimator_}")
print(f"The highest true positive score is {GB_cv.best_score_}")
print(f"The best estimator is {GB_cv.best_params_}")

The best estimator is GradientBoostingClassifier()
The highest true positive score is 0.02142857142857143
The best estimator is {'learning_rate': 0.1, 'n_estimators': 100}


In [ ]:
# performance_df.to_csv("230219-junior-test-data.csv")

In [ ]:
# cyclica_no_entry.to_csv("230219-cyclicaNoEntry.csv")